# 128 Measurement

To implement this part of face recognition, we need to import some specific packages. [Dlib](https://pypi.org/project/dlib) is a modern C++ toolkit containing machine learning algorithms and tools for creating complex software in C++ to solve real world problems. 

In [ ]:
import sys
import dlib
import cv2
import os
import glob

We first import the path of our project. Under the path, we have different files to save images, neural network model, face data of measurements and landmarks. You can build corresponding file path on your laptop and put the code, images, data in these files. After we build our path successfully, we can input our images, famous people`s photo is better. 

In [31]:
current_path = os.getcwd()  # get current path
print (current_path)

/Users/mac/7390/Final


In this part, we will import and load neural network of project. 

In [37]:
# path of model
predictor_path = current_path + "/model/shape_predictor_68_face_landmarks (1).dat"
face_rec_model_path = current_path + "/model/dlib_face_recognition_resnet_model_v1.dat"


In [38]:
#test images` path
faces_folder_path = current_path + "/faces"

In [39]:
# read and input model
detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor(predictor_path)
face_rec_model = dlib.face_recognition_model_v1(face_rec_model_path)

In this part, we convert image from BGR format to RGB format because we do not need the colored data to detect faces. Humans can easily recognize that both images are the same, but the computer thinks these two pictures are two completely different people. To solve this, we will try to distort each image so that the eyes and lips are always in the sample place in the image. This will make it easier to compare the differences between faces in the next steps.
To do this, we'll use an algorithm called face landmark estimation. There are many ways to do this, but this time we will use a method invented by Vahid Kazemi and Josephine Sullivan in 2014. The basic idea of this algorithm is to find 68 people face the prevalence of particular point (called feature points, the landmarks), including the top of the jaw, external contours of each eye, each internal contours of the eyebrows, etc. Next the program trains a machine learning algorithm that allows it to find 68 specific points in any face. We will place 68 feature points on each face. 


After we get 68 landmarks of face, we need to extract some basic measurements from each person's face. Then we can measure the unknown face in the same way and find the known face closest to the measurement. For example, we can measure the size of each ear, the distance between the eyes, the length of the nose, and so on. So the code is to train a deep convolutional neural network. However, it's not about identifying objects in the picture, and this time we're training it to generate 128 measurements for the face. And by comparing the 128 measurements from different images, we can find difference and get the same image.


In [ ]:
for img_path in glob.glob(os.path.join(faces_folder_path, "*.jpg")):
    print("Processing file: {}".format(img_path))
    # opencv read the image and display
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    # opencv convert image from BGR format to RGB format
    b, g, r = cv2.split(img)
    img2 = cv2.merge([r, g, b])
    dets = detector(img, 1)   # calibrat face
    print("Number of faces detected: {}".format(len(dets)))

    for index, face in enumerate(dets):
        print('face {}; left {}; top {}; right {}; bottom {}'.format(index, face.left(), face.top(), face.right(), face.bottom()))

        shape = shape_predictor(img2, face)   # capture 68 landmarks of face
        for i, pt in enumerate(shape.parts()):
            #print('Part {}: {}'.format(i, pt))
            pt_pos = (pt.x, pt.y)
            cv2.circle(img, pt_pos, 2, (255, 0, 0), 1)
            #print(type(pt))
        #print("Part 0: {}, Part 1: {} ...".format(shape.part(0), shape.part(1)))
        cv2.namedWindow(img_path+str(index), cv2.WINDOW_AUTOSIZE)
        cv2.imshow(img_path+str(index), img)

        face_descriptor = face_rec_model.compute_face_descriptor(img2, shape)   # calculate 128 vectors of face
        print(face_descriptor)

k = cv2.waitKey(0)
cv2.destroyAllWindows()

Processing file: /Users/mac/7390/Final/faces/Emma.jpg
Number of faces detected: 1
face 0; left 91; top 246; right 553; bottom 708
-0.161305
0.0652001
0.0167216
-0.0874412
-0.115376
-0.0352147
-0.038751
-0.090691
0.14881
-0.177061
0.169819
-0.0652394
-0.322522
0.047483
-0.0544668
0.145439
-0.193972
-0.122037
-0.0631586
-0.121744
0.0135362
0.0828123
0.0114669
0.104588
-0.140688
-0.300228
-0.1491
-0.101557
-0.128877
-0.0805732
0.0602865
0.0722952
-0.0997832
0.0112618
0.0747684
0.109706
-0.0203456
-0.08323
0.244281
-0.0157787
-0.169955
-0.00907224
0.205865
0.329943
0.268684
-0.00828258
-0.0172921
-0.00430323
0.195417
-0.326387
0.0101277
0.15433
0.157005
0.031648
0.131255
-0.135116
0.00602079
0.235568
-0.196762
0.0424563
0.0663703
-0.110236
0.0176481
0.000920264
0.295794
0.0458768
-0.232505
-0.0782014
0.232061
-0.219089
-0.0108243
0.126979
-0.167734
-0.276375
-0.222794
0.0883816
0.380695
0.177336
-0.0315785
0.114273
-0.0640652
-0.0391783
-0.0791077
0.131224
-0.0576154
-0.0402545
-0.0810987


# Comparison

In [3]:
import sys
import dlib
import cv2
import os
import glob
import numpy as np

In this part of code, we define a function for after use to compare two images. The paramter data1 and data2 are 128 vectors value of two images and we use euclidean distance to find the difference of two images. If the difference is higner than 0.6, they are not the same person. 

In [4]:
def comparePersonData(data1, data2):
    diff = 0
    # for v1, v2 in data1, data2:
        # diff += (v1 - v2)**2
    for i in range(len(data1)):
        diff += (data1[i] - data2[i])**2
    diff = np.sqrt(diff)
    print (diff)
    if(diff < 0.6):
        print ("It's the same person")
    else:
        print ("It's not the same person")

We save the 128 vectors of each image for future comapre.

In [26]:
def savePersonData(face_rec_class, face_descriptor):
    if face_rec_class.name == None or face_descriptor == None:
        return
    filePath = face_rec_class.dataPath + face_rec_class.name + '.npy'
    vectors = np.array([])
    for i, num in enumerate(face_descriptor):
        vectors = np.append(vectors, num)
        # print(num)
    print('Saving files to :'+filePath)
    
    np.save(filePath, vectors)
    return vectors

Load the face images

In [ ]:
def loadPersonData(face_rec_class, personName):
    if personName == None:
        return
    filePath = face_rec_class.dataPath + personName + '.npy'
    vectors = np.load(filePath)
    print(vectors)
    return vectors

Define the main class of code and initialize images

In [27]:
class face_recognition(object):
    def __init__(self):
        self.current_path = os.getcwd() # get current path
        self.predictor_path = self.current_path + "/model/shape_predictor_68_face_landmarks (1).dat"
        self.face_rec_model_path = self.current_path + "/model/dlib_face_recognition_resnet_model_v1.dat"
        self.faces_folder_path = self.current_path + "/faces"
        self.dataPath = self.current_path + "/data/"
        self.detector = dlib.get_frontal_face_detector()
        self.shape_predictor = dlib.shape_predictor(self.predictor_path)
        self.face_rec_model = dlib.face_recognition_model_v1(self.face_rec_model_path)

        self.name = None
        self.img_bgr = None
        self.img_rgb = None
        self.detector = dlib.get_frontal_face_detector()
        self.shape_predictor = dlib.shape_predictor(self.predictor_path)
        self.face_rec_model = dlib.face_recognition_model_v1(self.face_rec_model_path)

    def inputPerson(self, name='people', img_path=None):
        if img_path == None:
            print('No file!\n')
            return 

        # img_name += self.faces_folder_path + img_name
        self.name = name
        self.img_bgr = cv2.imread(self.current_path+img_path)
        # opencv convert image from BGR format to RGB format
        b, g, r = cv2.split(self.img_bgr)
        self.img_rgb = cv2.merge([r, g, b])

    def create128DVectorSpace(self):
        dets = self.detector(self.img_rgb, 1)
        print("Number of faces detected: {}".format(len(dets)))
        for index, face in enumerate(dets):
            print('face {}; left {}; top {}; right {}; bottom {}'.format(index, face.left(), face.top(), face.right(), face.bottom()))

            shape = self.shape_predictor(self.img_rgb, face)
            face_descriptor = self.face_rec_model.compute_face_descriptor(self.img_rgb, shape)
            # print(face_descriptor)
            # for i, num in enumerate(face_descriptor):
            #   print(num)
            #   print(type(num))

            return face_descriptor


In [28]:
face_rec = face_recognition()   # create object
face_rec.inputPerson(name="Emma", img_path="/faces/Emma.jpg")  # name中写第一个人名字，img_name为图片名字，注意要放在faces文件夹中
vector = face_rec.create128DVectorSpace()  # extract the 128-dimensional vector, which is the object of the dlib.vector class.
person_data1 = savePersonData(face_rec, vector )   # save the extracted data to data file，it is used to return numpy array，context is same

Number of faces detected: 1
face 0; left 91; top 246; right 553; bottom 708
Saving files to :/Users/mac/7390/Final/data/Emma.npy


In [14]:
# input the second image and extract feature vectors 
face_rec.inputPerson(name='Kerr', img_path='/faces/Kerr1.jpg')
vector = face_rec.create128DVectorSpace()  # extract 128 vectors，which is the object of the dlib.vector class
person_data2 = savePersonData(face_rec, vector )

Number of faces detected: 1
face 0; left 266; top 93; right 489; bottom 316
Saving files to :/Users/mac/7390/Final/dataKerr.npy


In [11]:
# calculate euclidean distance and judge if they are the same people
comparePersonData(person_data1, person_data2)

0.869438026398012
It's not the same person
